# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset - mongoimport --type json -d uk_food -c establishments --headerline --drop establishments.json

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
pprint(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
pprint(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
# review a document in the establishments collection
document = establishments.find_one()
pprint(document)

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishments.find(query)
for result in results:
    pprint(result)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessName": 0, "AddressLine1": 0,"AddressLine2": 0,"AddressLine3": 0,"AddressLine4": 0,
         "PostCode": 0,"Phone": 0,"LocalAuthorityCode": 0,"LocalAuthorityName": 0, "LocalAuthorityWebSite": 0,"LocalAuthorityEmailAddress": 0,
         "scores": 0,"SchemeType": 0,"geocode": 0,"RightToReply": 0,"Distance": 0,"NewRatingPending": 0,"FHRSID": 0,"ChangesByServerID": 0,
         "LocalAuthorityBusinessID": 0,"RatingValue": 0,"RatingKey": 0,"RatingDate": 0,"meta": 0,"links": 0,"_id": 0}
results = establishments.find(query, fields)
df = pd.DataFrame(results)
df.head(10)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set":{"BusinessTypeID":1}})

In [ ]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {"BusinessType": 0, "AddressLine1": 0,"AddressLine2": 0,"AddressLine3": 0,"AddressLine4": 0,
         "PostCode": 0,"Phone": 0,"LocalAuthorityCode": 0,"LocalAuthorityName": 0, "LocalAuthorityWebSite": 0,"LocalAuthorityEmailAddress": 0,
         "scores": 0,"SchemeType": 0,"geocode": 0,"RightToReply": 0,"Distance": 0,"NewRatingPending": 0,"FHRSID": 0,"ChangesByServerID": 0,
         "LocalAuthorityBusinessID": 0,"RatingValue": 0,"RatingKey": 0,"RatingDate": 0,"meta": 0,"links": 0,"_id": 0}
results = establishments.find(query, fields)
df = pd.DataFrame(results)
df.head()

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName":"Dover"}
establishments.delete_many(query)

In [ ]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query)

In [ ]:
# Check that other documents remain with 'find_one'
other_documents = establishments.find_one()
pprint(other_documents)


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}} ])

In [ ]:
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}} ])

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
])

In [ ]:
# Check that the coordinates and rating value are now numbers / WITH HELP FROM CHATGPT
query = {}
fields = {"geocode.latitude": 1,"geocode.longitude": 1,"RatingValue": 1, "_id" : 0}
limit = 5
results = establishments.find(query, fields, limit)

df = pd.DataFrame(results)

latitude_types = df["geocode"].apply(lambda x: type(x["latitude"])).unique()
longitude_types = df["geocode"].apply(lambda x: type(x["longitude"])).unique
rating_value_types = df["RatingValue"].apply(type).unique()
# Create a DataFrame
check_data_types = {
    "Latitude Types": latitude_types,
    "Longitude Types": longitude_types,
    "Rating Value Types": rating_value_types
}

df2 = pd.DataFrame(check_data_types)
pprint(df2)
# pprint(f"Types of latitude values: {latitude_types}")
# pprint(f"Types of longitude values: {longitude_types}")
# pprint(f"Types of RatingValue values: {rating_value_types}")